# Titanic: k-nearest-neighbor (KNN) from scratch [0.81339]

In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

**1. Data preparation**

First lets import the data and generate one set to process all data at once

In [14]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [15]:
train['set'], test['set'] = 'train', 'test'
combined = pd.concat([train, test])

Missing values

First step is to fill missing values and drop columns we will not be using.

In [16]:
combined.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
set               0
dtype: int64

We will not be using Embarked and Cabin for this experiment.

In [17]:
# combined = combined.drop(['Cabin', 'Embarked'], axis=1)

Only one Fare price is missing. fill it with the median of his Pclass:

In [18]:
pclass = combined.loc[combined.Fare.isnull(), 'Pclass'].values[0]
median_fare = combined.loc[combined.Pclass== pclass, 'Fare'].median()
combined.loc[combined.Fare.isnull(), 'Fare'] = median_fare

Missing ages

To fill in the missing ages, we can do something more clever then just take the overal median age. The names contain titles of which some are linked to their age. Master is a younger boy (in general). Lets take the median of each age group

In [19]:
# Select everything before the . as title
combined['Title'] = combined['Name'].str.extract('([A-Za-z]+)\.', expand=True)
combined['Title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'Countess',
       'Jonkheer', 'Dona'], dtype=object)

In [20]:
title_reduction = {'Mr': 'Mr', 'Mrs': 'Mrs', 'Miss': 'Miss', 
                   'Master': 'Master', 'Don': 'Mr', 'Rev': 'Rev',
                   'Dr': 'Dr', 'Mme': 'Miss', 'Ms': 'Miss',
                   'Major': 'Mr', 'Lady': 'Mrs', 'Sir': 'Mr',
                   'Mlle': 'Miss', 'Col': 'Mr', 'Capt': 'Mr',
                   'Countess': 'Mrs','Jonkheer': 'Mr',
                   'Dona': 'Mrs'}
combined['Title'] = combined['Title'].map(title_reduction)
combined['Title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Rev', 'Dr'], dtype=object)

In [22]:
for title, age in combined.groupby('Title')['Age'].median().items():
    print(title, age)
    combined.loc[(combined['Title']==title) & (combined['Age'].isnull()), 'Age'] = age

Dr 49.0
Master 4.0
Miss 22.0
Mr 30.0
Mrs 36.0
Rev 41.5


In [23]:
combined.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age               0
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin          1014
Embarked          2
set               0
Title             0
dtype: int64

**2. Create additional features**

An interesting theory I saw here on Kaggle is that there are groups of people, who would help each other. This might affect the survivability.

In [25]:
def other_family_members_survived(dataset, label='family_survival'):
    """
    Check if other family members survived
      -> 0 other did not survive
      -> 1 at least one other family member survived
      -> 0.5 unknown if other members survived or person was alone
    
    Parameters
    ----------
    dataset : DataFrame
      The sub-dataframe containing the family
    """
    ds = dataset.copy()
    if len(dataset) == 1:
        ds[label] = 0.5
        return ds
    result = []
    for ix, row in dataset.iterrows():
        survived_fraction = dataset.drop(ix)['Survived'].mean()
        if np.isnan(survived_fraction):
            result.append(0.5)
        elif survived_fraction == 0:
            result.append(0)
        else:
            result.append(1)
    ds[label] = result
    return ds

In [26]:
combined['surname'] = combined['Name'].apply(lambda x: x.split(",")[0])
combined = combined.groupby(['surname', 'Fare']).apply(other_family_members_survived).reset_index(drop=True)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_8924\2887954477.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  combined = combined.groupby(['surname', 'Fare']).apply(other_family_members_survived).reset_index(drop=True)


Missing data on families can also be extracted from Tickets. Same ticket orders have the same ticket number.

In [27]:
combined = combined.groupby(['Ticket']).apply(lambda x: other_family_members_survived(x, label='family_survival_ticket')).reset_index(drop=True)
combined.loc[combined['family_survival'] == 0.5, 'family_survival'] = combined.loc[combined['family_survival'] == 0.5, 'family_survival_ticket']

C:\Users\lenovo\AppData\Local\Temp\ipykernel_8924\2452194539.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  combined = combined.groupby(['Ticket']).apply(lambda x: other_family_members_survived(x, label='family_survival_ticket')).reset_index(drop=True)
